In [1]:
import requests
import pandas as pd
import math
import datetime
from tqdm import tqdm
import yfinance as yf
import exchange_calendars as ecals

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

from bs4 import BeautifulSoup

In [2]:
# Firebase Initialzation - 1번 실행

cred = credentials.Certificate('./firebase_key.json')
firebase_admin.initialize_app(cred, {
  'projectId': 'capstone-2022-07-dac76',
})

In [3]:
# 1번 실행
# 날짜 반영
# 만약 장중이 아니면 어제, 그저께로 쿼링해야 값들 가져올 수 있음
# 장중이면 오늘, 어제로 쿼링해야 실시간 반영 가능

XKRX = ecals.get_calendar("XKRX") # 한국 코드

current = datetime.datetime.now()
corr_current = current - datetime.timedelta(hours=9)

if(XKRX.is_trading_minute(corr_current.strftime("%Y-%m-%d %H:%M"))):
    previous = datetime.date.today().strftime("%Y-%m-%d")
    pre_previous = XKRX.previous_session(datetime.date.today().strftime("%Y-%m-%d")).strftime("%Y-%m-%d")
else:
    previous = XKRX.previous_session(datetime.date.today().strftime("%Y-%m-%d")).strftime("%Y-%m-%d")
    pre_previous = XKRX.previous_session(previous).strftime("%Y-%m-%d")

2022-04-29
20220428


In [23]:
# 1번 실행
# 주식 이름, 주식 코드, 시가총액 가져오는 함수
# 공공데이터포털 - 금융위원회_주식시세정보 api 켜야함

# 공공데이터포털 api 주소(Without param)
api_service_key_stock = [
    "RXhGWArdgsytKaKf0g%2FWxNuo27wXxg4iChLUs9ePc39VvneddFbQ9v9ZXCDWJkdFbhqCvbw9kdMGy%2F%2Bv3it50A%3D%3D",
    "bqvyeN8k%2B8%2BfRLf7p4CNQsUIEL%2BRb4b2YR08MD10RDv3BxHugq6bR1wFEAo8hTau3XgiLcA7bEBoclnMdyBfNQ%3D%3D",
    "zUgkw3obrruAXAW6kZrJnIpK8UUBIrwXrfroSgoDS7NUlSB%2BDz94OTIkkWeP0V%2BzOz81JVtW84bqh1y0HpzcUg%3D%3D",
    "w9Ra19Zqn3%2BLgg2zHoRiZa8zZPdSCXSgFgrgFGUkaYqqQRD6BVKMsUgiRyJqeEuG1pQ86vSioq03IRarAve7sg%3D%3D",
]  # service api key

code_list = []

# 종목 이름 가져오는 코드
def getStockCode(market, code_list):
    """
    market: 상장구분 (11=유가증권, 12=코스닥, 13=K-OTC, 14=코넥스, 50=기타비상장)
    """
    url = f"https://api.odcloud.kr/api/GetStockSecuritiesInfoService/v1/getStockPriceInfo?"
    stock_code = 0
    while True:
        api_decode_key_stock = requests.utils.unquote(
            api_service_key_stock[stock_code], encoding="utf-8"
        )

        params = {
            "serviceKey": api_decode_key_stock,
            "mrktCls": market,
            "numOfRows": 1000,
            "beginBasDt":pre_previous.replace("-", "")
        }

        response = requests.get(url, params=params)
        if(response.status_code != 200):
            print(response.status_code)
            stock_code += 1
            continue
        xml = BeautifulSoup(response.text, "lxml")
        items = xml.find("items")
        item_list = []
        for item in items:
            try:
                item_dict = {
                    "stockName": item.find("itmsnm").text.strip(),
                    "stockCode": item.find("srtncd").text.strip(),
                    "marketCap": item.find("mrkttotamt").text.strip(),
                }
                code_list.append(item.find("srtncd").text.strip() + ".KS")
            except AttributeError:
                continue
            item_list.append(item_dict)
            

        return item_list

In [24]:
item_list = getStockCode("KOSPI", code_list)

In [27]:
# Python3 샘플 코드 #


import requests

url = 'http://api.seibro.or.kr/openapi/service/StockSvc/getListStatN1'
params = {'serviceKey' : 'w9Ra19Zqn3%2BLgg2zHoRiZa8zZPdSCXSgFgrgFGUkaYqqQRD6BVKMsUgiRyJqeEuG1pQ86vSioq03IRarAve7sg%3D%3D',
          'pageNo' : '1',
          'numOfRows' : '1000',
          'stdDt' : pre_previous.replace("-", "") 
        }

response = requests.get(url, params=params)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>'


In [11]:
# 여기서부터는 시간별로 실행 - 실시간 코드

# start에는 장이 열리는 날 - 하루가 들어가야 함
data = yf.download(code_list, start=pre_previous)
print(data)

[*********************100%***********************]  0 of 0 completed


ValueError: No objects to concatenate

In [ ]:
# item_list에 각 필드들 쿼링

for j in tqdm(item_list):
    j["stockPrice"] = float(data["Close"][j["stockCode"] + ".KS"][previous])
    j["stockLowPrice"] = float(data["Low"][j["stockCode"] + ".KS"][previous])
    j["stockHighPrice"] = float(data["High"][j["stockCode"] + ".KS"][previous])
    j["stockVolume"] = float(data["Volume"][j["stockCode"] + ".KS"][previous])
    j["stockOpenPrice"] = float(data["Open"][j["stockCode"] + ".KS"][previous])
    j["stockClosingPrice"] = float(data["Close"][j["stockCode"] + ".KS"][pre_previous])
    j['stockChange'] = float(data['Close'][j["stockCode"] + ".KS"][previous] - data['Close'][j["stockCode"] + ".KS"][pre_previous])
    j['stockPerChange'] = float(round((data['Close'][j["stockCode"] + ".KS"][previous] - data['Close'][j["stockCode"] + ".KS"][pre_previous]) / data['Close'][j["stockCode"] + ".KS"][pre_previous] * 100, 2))

In [ ]:
# 상장폐지 / 거래중지 종목 리스트에서 제거

for i in item_list:
    if math.isnan(i['stockPrice']):
        item_list.remove(i)

In [ ]:
# Firebase에 multithreading 이용해서 쿼링
# 3분 30초 -> 20초

db = firestore.client()

from multiprocessing.dummy import Pool as ThreadPool
def AddFirebase(item):
    db.collection(u"stock_API").document(item["stockName"]).set(item)

pool = ThreadPool(10)

for _ in tqdm(pool.imap_unordered(AddFirebase, item_list), total=len(item_list)):
    pass

pool.close() 
pool.join()